<a href="https://colab.research.google.com/github/Sambhrama754/Crop-reccomender-ardiuno/blob/main/Crop_recommendation_Arduino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install micromlgen --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import pandas_profiling as pp
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
import pickle
from sklearn.utils import resample
# Metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, auc, roc_curve

# Validation
from sklearn.model_selection import train_test_split, cross_val_score, KFold

# Preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB

warnings.filterwarnings('ignore')
################################################################################
#                                                                              #
#                Split Data to Training and Validation set                     #
#                                                                              #
################################################################################
def read_in_and_split_data(data, target):
    X = data.drop(target, axis=1)
    y = data[target]
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.38, random_state=0)
    return X_train, X_test, y_train, y_test

################################################################################
#                                                                              #
#                           Train Model                                        #
#                                                                              #
################################################################################
def fit_model(X_train, y_train,models):
    # Test options and evaluation metric
    num_folds = 10
    scoring = 'accuracy'

    results = []
    names = []
    for name, model in models:
        kfold = KFold(n_splits=num_folds, shuffle=True, random_state=0)
        cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)
        
    return names, results
################################################################################
#                                                                              #
#                          Save Trained Model                                  #
#                                                                              #
################################################################################
def save_model(model,filename):
    pickle.dump(model, open(filename, 'wb'))
################################################################################
#                                                                              #
#                          Performance Measure                                 #
#                                                                              #
################################################################################
def classification_metrics(model, conf_matrix):
    print(f"Training Accuracy Score: {model.score(X_train, y_train) * 100:.1f}%")
    print(f"Validation Accuracy Score: {model.score(X_test, y_test) * 100:.1f}%")
    print(classification_report(y_test, y_pred))
    

# Load Dataset
df = pd.read_csv('https://raw.githubusercontent.com/Sambhrama754/Crop-reccomender-ardiuno/main/Crop_recommendation.csv')
# Remove Outliers
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
df_out = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]

# Split Data to Training and Validation set
target ='label'
X_train, X_test, y_train, y_test = read_in_and_split_data(df, target)

# Train model
Algo = GaussianNB()
model = Algo.fit(X_train, y_train)
y_pred = model.predict(X_test)
conf_matrix = confusion_matrix(y_test,y_pred)
classification_metrics(Algo, conf_matrix)

# save model
save_model(model, 'model.pkl')

Training Accuracy Score: 95.6%
Validation Accuracy Score: 95.9%
              precision    recall  f1-score   support

       apple       1.00      1.00      1.00        38
      banana       1.00      1.00      1.00        38
   blackgram       0.83      0.90      0.86        39
    chickpea       1.00      1.00      1.00        40
     coconut       1.00      0.97      0.99        36
      coffee       1.00      1.00      1.00        38
      cotton       0.95      1.00      0.97        35
      grapes       1.00      1.00      1.00        36
        jute       0.78      0.83      0.81        35
 kidneybeans       1.00      1.00      1.00        36
      lentil       0.81      0.91      0.86        33
       maize       1.00      0.95      0.97        38
       mango       1.00      1.00      1.00        32
   mothbeans       1.00      0.91      0.95        43
    mungbean       1.00      1.00      1.00        33
   muskmelon       1.00      1.00      1.00        38
      orange     

In [ ]:
classmapdat = {
     0: "Apple",
     1: "Banana",
     2: "Blackgram",
     3: "Chickpea",
     4: "Coconut",
     5: "Coffee",
     6: "Cotton",
     7: "Grapes",
     8: "Jute",
     9: "Kidneybean",
     10: "Lentil",
     11: "Maize",
     12: "Mango",
     13: "Mothbean",
     14: "Mungbean",
     15: "Muskmelon",
     16: "Orange",
     17: "Papaya",
     18: "Pigeonpeas",
     19: "Pomegranate",
     20: "Rice",
     21: "Watermelon"
}

In [ ]:
from micromlgen import port
Code = port(model, classmap=classmapdat)

In [ ]:
print(Code)

In [ ]:
import pickle
with open('model.h', 'wb') as f:
    pickle.dump(Code,f)